In [1]:
import os
import json
import fiona
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

from collections import Counter

from shapely.geometry import shape
from shapely.geometry import Point

# Gee and EE 
import ee

# Clustering (best for testing** due to speed)
from sklearn.cluster import KMeans

# tif file creation
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Plotting and Vis 
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import re

In [2]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [3]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [4]:
## TEMP REMOVE ADDITIONAL STUFF FOR QUICKER TESTING 
# tile_43641125_34108721_43915744_34336837 only 

chunks = chunks[chunks['file_name'] == 'tile_43641125_34108721_43915744_34336837']

In [5]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [6]:
def create_subregions(chunks, sift_percentage_lon=0.5, sift_percentage_lat=0.415):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount_lon = length_lon * sift_percentage_lon 
    shift_amount_lat = length_lon * sift_percentage_lat

    shift_directions = {
        "left": (-shift_amount_lon, 0),
        "right": (shift_amount_lon, 0),
        "up": (0, shift_amount_lat),
        "down": (0, -shift_amount_lat),
        "top_left": (-shift_amount_lon, shift_amount_lat),
        "top_right": (shift_amount_lon, shift_amount_lat),
        "bottom_left": (-shift_amount_lon, -shift_amount_lat),
        "bottom_right": (shift_amount_lon, -shift_amount_lat),
    }

    subregions = []
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

# Reference Points

In [7]:
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [8]:
# Example of 3 maps for urban, agricultrual, water and waste
urban_map = gen_basemap(basemap_url, aoi=chunks)
agricultural_map = gen_basemap(basemap_url, aoi=chunks)
water_map = gen_basemap(basemap_url, aoi=chunks)
wasteland_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_map = geemap.Map()

In [9]:
urban_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [10]:
agricultural_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# water_map = water_waste_map
water_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
wasteland_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
additional_map = gen_basemap(basemap_url, aoi=chunks)
additional_map


Map(center=[34.22282359393232, 43.778616017576226], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
additional_points = collect_points_from_geemap(additional_map, label="Urban")
additional_points

,geometry,label


In [15]:
# convert to labelled points in GeoDataFrame
water_points = collect_points_from_geemap(water_map, label="Water")
agricultural_points = collect_points_from_geemap(agricultural_map, label="Agricultural")
urban_points = collect_points_from_geemap(urban_map, label="Urban")
wasteland_map = collect_points_from_geemap(wasteland_map, label="Wasteland")

# merge into one GeoDataFrame
points_list = [water_points, agricultural_points, urban_points, wasteland_map]
labels = pd.concat(points_list, ignore_index=True)


# Clustering
Here we create a Sparse matrix

then with the sparse matrix compare and label based on that

In [16]:
pth = "data/"

# collect paths for all csv files in the folder
paths = []

for root, dirs, files in os.walk(pth):
    for file in files:
        if file.endswith(".csv"):
            paths.append(os.path.join(root, file))

print(f"Found {len(paths)} CSV files.")

# select 2024 
path = [p for p in paths if "2019" in p]
path


Found 6 CSV files.


['data/2019.csv']

In [17]:
def prep_data(dir):
    """Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact."""
    df1 = pd.read_csv(dir)
    df1 = df1.dropna()
    df2_clear = df1.copy()
    df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
    df2_clear = df2_clear.set_index("file_name")
    df1 = df1.set_index("file_name")
    df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

    return df1, df2_clear

df1, df2_clear = prep_data(path[0])

mapping = {
            None: 0,
            np.NaN: 0,
            "NaN": 0,
            "None": 0,
            "Water": 1,
            "Agricultural": 2,
            "Urban": 3,
            "Wasteland": 4,
        }

In [18]:
import cluster as cl 

In [ ]:
cluster = cl.Cluster(chunks, df1, points=labels, mapping=mapping, passes=6)
cluster.reload_state(filname_prefix="temp_test/test3", filename_postfix="_state")

,label,system:index,0_B1,0_B11,0_B12,0_B2,0_B3,0_B4,0_B5,0_B6,...,9_B2,9_B3,9_B4,9_B5,9_B6,9_B7,9_B8,9_B8A,9_B9,.geo
file_name,,,,,,,,,,,,,,,,,,,,,
tile_43997486_34112017_43998137_34112113,None,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,769.925866,3322.198887,2644.624125,1128.620894,1753.336923,2010.568300,2634.353258,3302.266379,...,1855.620740,2541.523502,3334.350796,3648.235249,3713.545965,3826.787522,3871.171013,3920.213786,3837.507039,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
tile_44104382_34112843_44105034_34112938,None,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,298.404905,2157.473569,1582.494005,528.488556,831.356676,934.839237,1352.791826,1881.520708,...,547.670409,801.668174,662.603379,1236.062398,2438.234332,2861.378801,3031.939292,3114.423978,2432.149682,"{""type"":""Polygon"",""coordinates"":[[[44.10438435..."
tile_43998136_34112022_43999221_34112122,None,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,528.454970,2495.875662,1713.649631,640.474810,1152.119767,1171.487171,1786.087047,3059.746338,...,1154.201264,1593.018742,2124.370565,2445.911989,2515.677113,2636.289053,2605.584472,2756.973423,3091.114321,"{""type"":""Polygon"",""coordinates"":[[[43.99813739..."
tile_43999220_34112031_44000305_34112130,None,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,335.551290,2071.815893,1344.805779,429.478019,894.811249,770.613622,1463.251290,2839.394324,...,1018.777621,1394.102079,1854.067183,2060.790668,2105.297921,2202.409155,2225.273773,2299.559797,2382.651482,"{""type"":""Polygon"",""coordinates"":[[[43.99922097..."
tile_44000304_34112040_44001389_34112139,None,1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_1_...,285.086948,1878.116894,1177.896530,406.417493,840.937113,721.051941,1302.629802,2778.296159,...,1015.020712,1386.387060,1850.829881,2059.086564,2108.126792,2208.764339,2278.053815,2330.043238,2451.489202,"{""type"":""Polygon"",""coordinates"":[[[44.00030452..."


In [ ]:
class ts_cluster():

    yearly_clusters = []

    def __init__(self, ts_point_labels, data_dir, mapping, passes=6):
        """
            Initializes the ts_cluster object with time-series point labels, data directory, mapping, and number of passes.
        """
        self.ts_point_labels = ts_point_labels          # expected labels in format year_start, year_end, label and point geometry 
        self.data_dir = data_dir                        # expected to have file_dir and year columns in pd.DataFrame format  
        self.mapping = mapping                          # mapping for labels to numeric values 
        self.passes = passes                            # int 
    

    def __create_shared_label_list__(self):
        """
            Creates a list of shared labels acrross ALL time-series points, such that it can act as a base for labels in the clustering process.
        """

        shared_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])
        non_shared_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])

        min_year = self.data_dir['year'].min()
        max_year = self.data_dir['year'].max()

        # if years between (incluside) are in ts_point_labels, then add to shared_labels else add to non_shared_labels
        for year in range(min_year, max_year + 1):
            if year in self.ts_point_labels['year_start'].values:
                shared_labels = shared_labels.append(self.ts_point_labels[self.ts_point_labels['year_start'] == year], ignore_index=True)

        # else 
        non_shared_labels = self.ts_point_labels[~self.ts_point_labels['year_start'].isin(shared_labels['year_start'])]

    